# NoVaS

This notebook contains the results of voltality forecasting with the NoVaS transformation (see Politis 2007)on S&P500 returns.

Need to implement:
- Simple NoVaS
- Exponential NoVaS



%% latex
Regular ARCH model is expressed as $$ \frac{X_t}{sqrt{a + \sum_{i=1}^{p} a_i X^{2}_{t-i} $$

In [1]:
%% latex

UsageError: Cell magic `%%` not found.
